In [ ]:
import json
import os
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/youtube')

In [ ]:
video_df = pd.read_csv('USvideos.csv')

In [ ]:
video_df

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09
...,...,...,...,...,...,...,...,...,...,...,...
7995,xlu6i6lT_vk,How Do MASSIVE Sinkholes Form?,Life Noggin,27,sinkhole|how do sinkholes form|sinkhole in wat...,440393,14362,390,1575,https://i.ytimg.com/vi/xlu6i6lT_vk/default.jpg,22.10
7996,qRoVlH1OcI4,Trump slams Clinton for defending NFL anthem p...,Business Insider,25,Business Insider|Donald Trump|Hillary Clinton|...,55762,1265,760,1873,https://i.ytimg.com/vi/qRoVlH1OcI4/default.jpg,22.10
7997,EoejGgUNmVU,LP - Lost On You (A Night at The McKittrick Ho...,LP,10,LP|Death Valley|Other People|Lost On You|The M...,142908,7088,68,437,https://i.ytimg.com/vi/EoejGgUNmVU/default.jpg,22.10
7998,MT1CMTI0EVw,Tré Melvin @ #YouTubeBlack FanFest Washington ...,YouTube FanFest,24,YouTube FanFest|#YTFF|Washington DC|USA|YTFF|#...,24532,2148,77,0,https://i.ytimg.com/vi/MT1CMTI0EVw/default.jpg,22.10


In [ ]:
comments_df = pd.read_csv('UScomments 2.csv', usecols=['video_id', 'comment_text', 'likes', 'replies'],encoding_errors='ignore')
comments_df['likes'] = pd.to_numeric(comments_df['likes'], errors='coerce')
comments_df['replies'] = pd.to_numeric(comments_df['replies'], errors='coerce')
comments_df = comments_df.dropna(subset=['likes', 'replies'])
comments_df = comments_df.drop_duplicates(subset=['video_id', 'comment_text'])
comments_df['comment_weight'] = comments_df['likes'] + comments_df['replies']

<ipython-input-6-f955392cf6b0>:1: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  comments_df = pd.read_csv('UScomments 2.csv', usecols=['video_id', 'comment_text', 'likes', 'replies'],encoding_errors='ignore')
<ipython-input-6-f955392cf6b0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments_df['comment_weight'] = comments_df['likes'] + comments_df['replies']


In [ ]:
comments_df

,video_id,comment_text,likes,replies,comment_weight
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4.0,0.0,4.0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3.0,0.0,3.0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3.0,0.0,3.0
3,XpVt6Z1Gjjo,MY FAN . attendance,3.0,0.0,3.0
4,XpVt6Z1Gjjo,trending 😉,3.0,0.0,3.0
...,...,...,...,...,...
691368,EoejGgUNmVU,Lost on you ! 😍\nOH My God I love this song !😭,0.0,0.0,0.0
691375,EoejGgUNmVU,I LOVE LP❤️,0.0,0.0,0.0
691401,EoejGgUNmVU,*L O S T O N Y O U*,1.0,0.0,1.0
691402,EoejGgUNmVU,<3,0.0,0.0,0.0


In [ ]:
with open('US_category_id.json', 'r') as file:
    data = file.read()
    categories = json.loads(data)


In [ ]:
comments_more_than_one_word = comments_df[comments_df['comment_text'].str.split().str.len() > 1]

In [ ]:
#merging two dfs on video_id (converted to string)
comments_more_than_one_word['video_id'] = comments_more_than_one_word['video_id'].astype(str)
video_df['video_id'] = video_df['video_id'].astype(str)
merged_df = pd.merge(comments_more_than_one_word, video_df, on = 'video_id', how = 'inner')
merged_df = merged_df.drop_duplicates(subset=['video_id', 'comment_text']).drop(columns = ['comment_total', 'views', 'date'])

<ipython-input-10-b5e5c8bc9448>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments_more_than_one_word['video_id'] = comments_more_than_one_word['video_id'].astype(str)


In [ ]:
merged_df

,video_id,comment_text,likes_x,replies,comment_weight,title,channel_title,category_id,tags,likes_y,dislikes,thumbnail_link
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4.0,0.0,4.0,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,320053,5931,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg
7,XpVt6Z1Gjjo,I've been following you from the start of your...,3.0,0.0,3.0,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,320053,5931,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg
14,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3.0,0.0,3.0,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,320053,5931,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg
21,XpVt6Z1Gjjo,MY FAN . attendance,3.0,0.0,3.0,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,320053,5931,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg
28,XpVt6Z1Gjjo,trending 😉,3.0,0.0,3.0,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,320053,5931,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
1758925,EoejGgUNmVU,Thank so much for you song grant,0.0,0.0,0.0,LP - Lost On You (A Night at The McKittrick Ho...,LP,10,LP|Death Valley|Other People|Lost On You|The M...,5763,44,https://i.ytimg.com/vi/EoejGgUNmVU/default.jpg
1758929,EoejGgUNmVU,Lost on you ! 😍\nOH My God I love this song !😭,0.0,0.0,0.0,LP - Lost On You (A Night at The McKittrick Ho...,LP,10,LP|Death Valley|Other People|Lost On You|The M...,5763,44,https://i.ytimg.com/vi/EoejGgUNmVU/default.jpg
1758933,EoejGgUNmVU,I LOVE LP❤️,0.0,0.0,0.0,LP - Lost On You (A Night at The McKittrick Ho...,LP,10,LP|Death Valley|Other People|Lost On You|The M...,5763,44,https://i.ytimg.com/vi/EoejGgUNmVU/default.jpg
1758937,EoejGgUNmVU,*L O S T O N Y O U*,1.0,0.0,1.0,LP - Lost On You (A Night at The McKittrick Ho...,LP,10,LP|Death Valley|Other People|Lost On You|The M...,5763,44,https://i.ytimg.com/vi/EoejGgUNmVU/default.jpg


In [ ]:
# !pip install fasttext
!pip install swifter
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
!pip install langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=2c8e1ef414213ba15c5e53e3809dfb7be0ff207ade457fe4db1c520da83e4b03
  Stored in directory: /root/.cache/pip/wheels/ef/7f/bd/9bed48f078f3ee1fa75e0b29b6e0335ce1cb03a38d3443b3a3
Successfully built swifter
--2025-04-06 20:53:53--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.102, 108.157.254.124, 108.157.254.121, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: ‘lid.176.bin.5’

lid.176.bin.5       100%[===================>] 125.18M  91.6MB/s    in 1.4s    

2025-04-06 20:53:54 (91.6 MB/s) - ‘lid.176.bin.5’ saved [131266198/131266198]


In [ ]:
#filter out non-English comments
!pip install langid
!pip install swifter
import langid
import swifter

def is_english_fast(text):
    if isinstance(text, str) and len(text.strip()) > 0:
        lang, _ = langid.classify(text)
        return lang == 'en'
    return False

merged_df['is_english'] = merged_df['comment_text'].swifter.apply(is_english_fast)
merged_english = merged_df[merged_df['is_english']]

Pandas Apply:   0%|          | 0/420104 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
merged_english = pd.merged_english.csv

In [ ]:
#convert category id to names, keep the top 10 most common
category_counts = merged_english['category_name'].value_counts()
top_categories = category_counts[category_counts >= 10000].head(10).index
merged_english_filtered = merged_english[merged_english['category_name'].isin(top_categories)]
merged_english_filtered

#clean comment texts like removing /n, links, non-punctuation symbols, etc.
import re
import string

def clean_text(text):
    text = text.lower()
    text = text.replace('\n', '')
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    #text = re.sub(r'[^a-zA-Z0-9\s.,!?\'"]', '', text)
    text = re.sub(r'[^\w\s.,!?\'\"😃-🙏❤🧠🔥😭💀😅😉🤣🤡🤠😎🥲🥺🤔👀⭐🌟🌈🎉✨❤️🫶🤭😭❤️😍👍]+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

merged_english_filtered['comment_text'] = merged_english_filtered['comment_text'].apply(clean_text)
merged_english_filtered

<ipython-input-46-ee1ee2892702>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_english_filtered['comment_text'] = merged_english_filtered['comment_text'].apply(clean_text)


,video_id,comment_text,likes_x,replies,comment_weight,title,channel_title,category_id,tags,likes_y,dislikes,thumbnail_link,is_english,clean_tags,category_name
7,XpVt6Z1Gjjo,i've been following you from the start of your...,3.0,0.0,3.0,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,320053,5931,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,True,"[logan paul vlog, logan paul, logan, paul, oly...",Entertainment
14,XpVt6Z1Gjjo,say hi to kong and maverick for me,3.0,0.0,3.0,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,320053,5931,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,True,"[logan paul vlog, logan paul, logan, paul, oly...",Entertainment
35,XpVt6Z1Gjjo,1 on trending ayyeeeee,3.0,0.0,3.0,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,320053,5931,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,True,"[logan paul vlog, logan paul, logan, paul, oly...",Entertainment
42,XpVt6Z1Gjjo,the end though 😭👍❤️,4.0,0.0,4.0,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,320053,5931,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,True,"[logan paul vlog, logan paul, logan, paul, oly...",Entertainment
49,XpVt6Z1Gjjo,1 trending!!!!!!!!!,3.0,0.0,3.0,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,320053,5931,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,True,"[logan paul vlog, logan paul, logan, paul, oly...",Entertainment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1758909,xlu6i6lT_vk,present maybe everything surprisingly dose ter...,0.0,0.0,0.0,How Do MASSIVE Sinkholes Form?,Life Noggin,27,sinkhole|how do sinkholes form|sinkhole in wat...,10792,255,https://i.ytimg.com/vi/xlu6i6lT_vk/default.jpg,True,"[sinkhole, how do sinkholes form, sinkhole in ...",Education
1758915,qRoVlH1OcI4,when you take a knee you dont get elected,0.0,0.0,0.0,Trump slams Clinton for defending NFL anthem p...,Business Insider,25,Business Insider|Donald Trump|Hillary Clinton|...,456,251,https://i.ytimg.com/vi/qRoVlH1OcI4/default.jpg,True,"[business insider, donald trump, hillary clint...",News & Politics
1758921,EoejGgUNmVU,u are an angel,0.0,0.0,0.0,LP - Lost On You (A Night at The McKittrick Ho...,LP,10,LP|Death Valley|Other People|Lost On You|The M...,5763,44,https://i.ytimg.com/vi/EoejGgUNmVU/default.jpg,True,"[lp, death valley, other people, lost on you, ...",Music
1758925,EoejGgUNmVU,thank so much for you song grant,0.0,0.0,0.0,LP - Lost On You (A Night at The McKittrick Ho...,LP,10,LP|Death Valley|Other People|Lost On You|The M...,5763,44,https://i.ytimg.com/vi/EoejGgUNmVU/default.jpg,True,"[lp, death valley, other people, lost on you, ...",Music


In [ ]:
#clean tags
merged_english['clean_tags'] = (
    merged_english['tags']
    .fillna('')
    .str.lower()
    .str.split('|')
    .apply(lambda x: list(dict.fromkeys([tag.strip() for tag in x if tag.strip()])))
)
merged_english

NameError: name 'merged_english' is not defined

In [ ]:
from collections import Counter

tag_list = [tag for tag_list_row in merged_english['clean_tags'].dropna() for tag in tag_list_row]

tag_counts = Counter(tag_list)

excluded_tags = {'[none]', '', 'none'}

filtered_tag_counts = {tag: count for tag, count in tag_counts.items() if tag.lower() not in excluded_tags}

top_10_tags = Counter(filtered_tag_counts).most_common(10)

top_10_tags


In [ ]:
category_dict = {
    1: "Film & Animation",
    2: "Autos & Vehicles",
    10: "Music",
    15: "Pets & Animals",
    17: "Sports",
    18: "Short Movies",
    19: "Travels and Events",
    20: "Gaming",
    21: "Videoblogging",
    22: "People & Blogs",
    23: "Comedy",
    24: "Entertainment",
    25: "News & Politics",
    26: "Howto & Style",
    27: "Education",
    28: "Science & Technology",
    29: "Nonprofits & Activism",
    30: "Movies",
    31: "Anime/Animation",
    32: "Action/Adventure",
    33: "Classics",
    34: "Comedy",
    35: "Documentary",
    36: "Drama",
    37: "Family",
    38: "Foreign",
    39: "Horror",
    40: "Sci-Fi/Fantasy",
    41: "Thriller",
    42: "Shorts",
    43: "Shows",
    44: "Trailers"
}
merged_english['category_name'] = merged_english['category_id'].map(category_dict).drop(columns = ['category_id'])

In [ ]:
merged_english['comment_weight'].dtype

# Demo

In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st

In [ ]:
st.title("Random Comment Generator with Models")

# Selectbox to choose a model
model_choice = st.selectbox("Choose a model", ["Markov Model 1", "Markov Model 2"])


## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline

In [ ]:
tfidf_small = TfidfVectorizer(max_features=20000, ngram_range=(1,2))


In [ ]:
base_model_small = Pipeline([
    ('tfidf', tfidf_small),
    ('classifier', LogisticRegression(max_iter = 2000, class_weight = 'balanced'))
])

In [ ]:
weighted_model_small = Pipeline([
    ('tfidf', tfidf_small),
    ('classifier', LogisticRegression(max_iter = 2000))
])

In [ ]:
X = merged_english['comment_text']
y = merged_english['category_name']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
base_model_small.fit(X_train, y_train)
y_pred_base = base_model_small.predict(X_test)
print("=== Baseline Model (Unweighted TF-IDF) ===")
print(classification_report(y_test, y_pred_base))

=== Baseline Model (Unweighted TF-IDF) ===
                       precision    recall  f1-score   support

     Autos & Vehicles       0.14      0.51      0.21       774
               Comedy       0.43      0.32      0.37      8821
            Education       0.26      0.44      0.33      2990
        Entertainment       0.61      0.23      0.33     15630
     Film & Animation       0.32      0.46      0.38      3732
               Gaming       0.12      0.43      0.18       721
        Howto & Style       0.57      0.54      0.55      8966
                Music       0.60      0.58      0.59     10551
      News & Politics       0.35      0.52      0.42      4509
Nonprofits & Activism       0.06      0.44      0.11         9
       People & Blogs       0.47      0.28      0.35      8571
       Pets & Animals       0.17      0.55      0.26      1029
 Science & Technology       0.44      0.46      0.45      5074
                Shows       0.00      0.00      0.00         3
           

In [ ]:
weighted_model_small.fit(X_train, y_train, classifier__sample_weight=merged_english.loc[X_train.index, 'comment_weight'])
y_pred_weighted_base = weighted_model_small.predict(X_test)
print("\n=== Weighted Model ===")
print(classification_report(y_test, y_pred_weighted_base))

In [ ]:
models = {
    '5K': Pipeline([('tfidf', TfidfVectorizer(max_features=5000)), ('classifier', LogisticRegression(max_iter = 2000, class_weight = 'balanced'))]),
    '10K': Pipeline([('tfidf', TfidfVectorizer(max_features=10000)), ('classifier', LogisticRegression(max_iter = 2000, class_weight = 'balanced'))]),
    '20K': Pipeline([('tfidf', TfidfVectorizer(max_features=20000)), ('classifier', LogisticRegression(max_iter = 2000, class_weight = 'balanced'))])
}


for name, model in models.items():
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print(f"{name} features: Accuracy = {score:.3f}")

5K features: Accuracy = 0.368
10K features: Accuracy = 0.408
20K features: Accuracy = 0.429


## Markov Model

In [ ]:
!pip install markovify
!pip install transformers

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 18.8 MB/s eta 0:00:00
  Created wheel for markovify: filename=markovify-0.9.4-py3-none-any.whl size=18606 sha256=a66b282e22cc898f280dd8c04ad8c52d55c3d4b8251230c4942ec0454607c0fc
  Stored in directory: /root/.cache/pip/wheels/9c/20/eb/1a3fb93f3132f2f9683e4efd834800f80c53aeddf50e84ae80
Successfully built markovify


In [ ]:
import markovify
import numpy as np
from transformers import pipeline, set_seed
music_comments = merged_english_filtered[merged_english_filtered['category_name'] == 'Music']['comment_text']
text_model = markovify.Text(' '.join(music_comments), state_size=4)

generator = pipeline('text-generation', model='gpt2')
set_seed(42)

prompt = "Category: Music | Mood: Hype | Comment:"
generator(prompt, max_length=50, num_return_sequences=5)

#keep comments with most 10 common tags
top_10_tags_set = set([tag for tag, _ in top_10_tags])
merged_english_filtered_on_both = merged_english_filtered[
    merged_english_filtered['clean_tags'].apply(lambda tag_list: any(tag in top_10_tags_set for tag in tag_list))
]
merged_english_filtered_on_both

#GPT-2 Transformer Generator using both category and tags
generator = pipeline('text-generation', model='gpt2')
set_seed(42)

def generate_youtube_comments(category, tags, mood="Hype", num_comments=5):
    prompt = f"Category: {category} | Tags: {tags} | Mood: {mood} | Comment:"

    outputs = generator(
        prompt,
        max_length=50,
        num_return_sequences=num_comments,
        do_sample=True,
        top_k=50,
        temperature=0.9
    )

    return [out['generated_text'] for out in outputs]

# Test example
category = "Music"
tags = "funny, vlog, humor"
mood = "Hype"

print("🎤 GPT-2 YouTube Comments for Music + Funny Tags:")
comments = generate_youtube_comments(category, tags, mood)

for i, comment in enumerate(comments, 1):
    print(f"{i}. {comment}")


ModuleNotFoundError: No module named 'markovify'

## Trigram

In [ ]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
nltk.download('punkt')

comments = merged_english_filtered.copy()
comments = comments['comment_text'].dropna().tolist()

tokens = []
for comment in comments:
    tokens.extend(word_tokenize(comment.lower()))
Trigrams = list(ngrams(tokens, 3))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Error loading punk_tab: Package 'punk_tab' not found in
[nltk_data]     index
